In [23]:
import pandas as pd
import numpy as np
import os, glob,re

from tqdm import tqdm
import matplotlib.pyplot as plt


import torch
PATH = '/content/drive/MyDrive/Coleridge_Initiative/input'

In [ ]:
!pip install transformers

In [25]:
import transformers

In [10]:
for dir, folder, files in os.walk(PATH, topdown=False):
    print(dir,folder,files)

/content/drive/MyDrive/Coleridge_Initiative/input/train [] ['f0249680-2273-4b1a-8a0a-2e8bbf9ec11a.json', 'ef4b7964-b7dc-4570-804e-74cc7b804c61.json', 'ec938f38-37e7-4a35-87bd-e59ffa02d52e.json', 'f042f6d8-c940-4cda-9b96-75cb95db4ef8.json', 'ed1a995e-63f9-4542-b515-bcaab0ed4295.json', 'eaf60533-ad57-4a1a-9f3b-6694206db77b.json', 'e9a1464e-bdbc-41cb-b331-4122b49df737.json', 'ebd1f6d3-9603-43ce-8c1c-7c80ed5931e3.json', 'edfed605-ef59-4d2f-b895-ae5d923a2598.json', 'e972d82c-2349-434d-a1ad-8ee16145c4f6.json', 'ef543aa8-230a-4efc-976f-754ae86dd8a6.json', 'eeffe52c-e93b-4778-a7dd-e9ecd5e8e366.json', 'e8dddd88-5804-49f9-a80e-883fcabb2c98.json', 'f03ccb15-f7f7-479f-adee-a0e49c443042.json', 'e96dec31-9b22-4ff3-bbee-c40d86816bad.json', 'ece88509-61ce-4874-99fe-15828041fbdf.json', 'e9767ab5-0419-4d99-97ff-a08cf5dbab6f.json', 'e7ed9cac-fd1d-4202-afce-e95b285cd8c2.json', 'ef06f1a9-a7b7-447f-b4e5-4eb00ef9a860.json', 'ec337fbe-7449-474e-867d-4955a0d873c3.json', 'edb7e856-3223-47f3-935c-3ba77c2faadd.js

In [11]:
df = pd.read_csv(os.path.join(PATH, 'train.csv'))
df.head()

Id  ...                          cleaned_label
0  d0fa7568-7d8e-4db9-870f-f9c6f668c17b  ...  national education longitudinal study
1  2f26f645-3dec-485d-b68d-f013c9e05e60  ...  national education longitudinal study
2  c5d5cd2c-59de-4f29-bbb1-6a88c7b52f29  ...  national education longitudinal study
3  5c9a3bc9-41ba-4574-ad71-e25c1442c8af  ...  national education longitudinal study
4  c754dec7-c5a3-4337-9892-c02158475064  ...  national education longitudinal study

[5 rows x 5 columns]

Look for text


In [14]:
df1 = pd.read_json(os.path.join(PATH, 'train', 'd0fa7568-7d8e-4db9-870f-f9c6f668c17b.json'))
df1

section_title                                               text
0              What is this study about?  This study used data from the National Educati...
1   Features of Dual Enrollment Programs  Dual enrollment programs allow high school stu...
2                WWC Single Study Review  What did the study find?\nThe study reported, ...
3                             WWC Rating  The research described in this report meets WW...
4                     Intervention group  The intervention group was comprised of those ...
5                       Comparison group  The comparison group was comprised of NELS par...
6               Outcomes and measurement  There were two outcomes examined in the study:...
7             Support for implementation  The study was based on secondary data from a n...
8                      Reason for review  This study was identified for review by receiv...
9                      Degree attainment  Any college degree attainment The study author...
10          Bachelor's degree attainment  The study author collected information on bach...
11                          Study Notes:  A correction for multiple comparisons was need...
12                     Glossary of Terms  Attrition Attrition occurs when an outcome var...
13        Multiple comparison adjustment  When a study includes multiple outcomes or com...
14       Quasi-experimental design (QED)  A quasi-experimental design (QED) is a researc...
15     Randomized controlled trial (RCT)  A randomized controlled trial (RCT) is an expe...
16              Single-case design (SCD)  A research approach in which an outcome variab...
17                    Standard deviation  The standard deviation of a measure shows how ...

for BERT not useble very length text, base model 512 tokens

In [19]:
df1.text.apply(lambda x: len(x.split())).describe()

count     18.000000
mean      98.000000
std       95.736157
min       10.000000
25%       27.750000
50%       41.500000
75%      165.000000
max      298.000000
Name: text, dtype: float64

In [26]:
len_text = len(df1.text.str.cat(sep = '\n').lower().split())
len_text

1764

In [29]:
txt = df1.text.str.cat(sep = '\n').lower()
txt[:100]

'this study used data from the national education longitudinal study (nels:88) to examine the effects'

In [30]:
model_name = 'bert-base-uncased'
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
tokenizer

PreTrainedTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [58]:
txt = 'this study used data from the national education longitudinal study (nels:88) to examine the effects'
out_dict = tokenizer.encode_plus(
    txt,
    add_special_tokens = True,
    max_length = 24,
    padding = 'max_length',
    return_tensors = 'pt',
    truncation = True,
    return_attention_mask = True
)

In [59]:
out_dict

{'input_ids': tensor([[  101,  2023,  2817,  2109,  2951,  2013,  1996,  2120,  2495, 20134,
          2817,  1006, 11265,  4877,  1024,  6070,  1007,  2000, 11628,  1996,
          3896,   102,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0]])}

In [60]:
voc = tokenizer.get_vocab()
voc = {v: k for (k, v) in voc.items()}

In [61]:
[voc[i] for i in out_dict['input_ids'].tolist()[0]]

['[CLS]',
 'this',
 'study',
 'used',
 'data',
 'from',
 'the',
 'national',
 'education',
 'longitudinal',
 'study',
 '(',
 'ne',
 '##ls',
 ':',
 '88',
 ')',
 'to',
 'examine',
 'the',
 'effects',
 '[SEP]',
 '[PAD]',
 '[PAD]']

In [68]:
len(df.Id.unique())

14316

не могу сообразить что указывать в таргет

в каждом json есть заголовки может исрользовать только их


передаем json и читаем по заголовкам
к примеру у нас в d0fa7568-7d8e-4db9-870f-f9c6f668c17b.json 18 заголовков в каждом текст
есть ли смысл читать и по каждому что то вернуть а потом оптимизировать(тк усреднить)

искать кааой-то контекст по заголовку но их много

может сделать ответ на вопрос или поиск сущности

[Summary of the tasks](https://huggingface.co/transformers/task_summary.html#)

try:
- Masked Language Modeling
- Extractive Question Answering
- Summarization


In [ ]:
https://huggingface.co/transformers/preprocessing.html
https://huggingface.co/transformers/tokenizer_summary.html

In [ ]:
https://medium.com/@aniruddha.choudhury94/part-2-bert-fine-tuning-tutorial-with-pytorch-for-text-classification-on-the-corpus-of-linguistic-18057ce330e1